# Dev / Library

In [1]:
import json
def open_json(file_path):
    datastore = {}
    with open(file_path, 'r') as f:
        datastore = json.load(f)
    return datastore

def import_to_name(library_import, depth):
    return '.'.join(library_import.split('.')[:depth])

def define_mapping(libraries_set, depth):
    A = set(sorted([import_to_name(library_import, depth) for library_import in libraries_set]) + ['UNK'])
    length = len(A)
    map_dict = dict(zip(A, range(length)))
    def mapping(import_value):
        name = import_to_name(import_value, depth)
        if name not in A:
            name = 'UNK'
        return map_dict[name]
    return mapping, A, length

# 2 more or less equal to the library level 
data = open_json('spoon_commits.json')
libraries_set = data['libraries_set']
commit_info = data['commit_info']
mapping, mapping_elemnts, mapping_size = define_mapping(libraries_set, 2)

In [2]:
def v_sum(*v):
    return tuple([sum(e) for e in zip(*v)])

def z_k(n, *Ks):
    z_k = [0] * n
    for i in Ks:
        z_k[i] = 1
    return tuple(z_k)

v_sum((1,2,3), (0,1,0), (5,0,0,10))

(6, 3, 3)

In [3]:
import pandas as pd
data = []

for sha, commit in list(commit_info.items()):
    libraries = set()
    for file, file_info in commit['files_info'].items():
        libraries = libraries | set([ mapping(lib) for lib in file_info['imports'] ])
    if len(libraries) > 0:
        data.append( [sha, commit['author']['name']] + list(z_k(mapping_size, *libraries)))
  
#data
df = pd.DataFrame(data=data, columns=['sha', 'author', *mapping_elemnts])

In [4]:
agreg = df.groupby('author').sum()

In [5]:
%%capture 
!pip install plotly

In [6]:
import plotly.plotly as py
import numpy as np
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Layout, Heatmap, Figure
import plotly.figure_factory as ff

#configure_plotly_browser_state()
init_notebook_mode(connected=True)

def plot_matrix(df, title, xlabel = '', ylabel=''):
    data = [Heatmap( z=df.values.tolist(), y=df.index, x=list(df.columns), colorscale='Viridis')]
    layout = Layout(
        title=title,
        xaxis=dict(
            title=xlabel
        ),
        yaxis=dict(
            title=ylabel
        )
    )
    fig = Figure(data=data, layout=layout)
    iplot(fig)

In [7]:
plot_matrix(agreg, 'number of contributions', xlabel='Library', ylabel='Dev')

In [8]:
plot_matrix(agreg.div(agreg.sum(axis=1), axis = 0), 'number of contributions / contibution by author', xlabel='Library', ylabel='Dev')

In [9]:
plot_matrix(agreg.div(agreg.sum(axis=0), axis = 1), 'number of contributions / contibution by library', xlabel='Library', ylabel='Dev')